In [ ]:
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, Image

def fetch_books(query, max_results=40):
    search_url = f"https://openlibrary.org/search.json?q={query}&limit={max_results}"
    response = requests.get(search_url)
    
    if response.status_code != 200:
        print("Failed to fetch data from Open Library API.")
        return None
    
    data = response.json()
    books = []
    
    for doc in data.get('docs', []):
        title = doc.get('title', 'Unknown Title')
        cover_id = doc.get('cover_i')
        image = f"https://covers.openlibrary.org/b/id/{cover_id}-M.jpg" if cover_id else None
        key = doc.get('key', '')
        
        description = ''
        if key:
            works_url = f"https://openlibrary.org{key}.json"
            works_response = requests.get(works_url)
            if works_response.status_code == 200:
                works_data = works_response.json()
                description = works_data.get('description', '')
                if isinstance(description, dict):
                    description = description.get('value', '')

        books.append({
            'title': title,
            'description': description,
            'image': image
        })
    
    return pd.DataFrame(books)

def get_recommendations(book_title, books_df):
    if book_title not in books_df['title'].values:
        return "Book title not found in the dataset"
    
    books_df['description'] = books_df['description'].fillna('')
    
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(books_df['description'])
    
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    idx = books_df[books_df['title'] == book_title].index[0]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    book_indices = [i[0] for i in sim_scores]
    
    return books_df.iloc[book_indices][['title', 'image']]

query = "To Kill a Mockingbird"
books_df = fetch_books(query)
if books_df is not None:
    display(books_df.head())

if books_df is not None:
    book_title = "To Kill a Mockingbird"
    recommended_books = get_recommendations(book_title, books_df)
    
    if isinstance(recommended_books, str):
        print(recommended_books)
    else:
        print("Recommended Books:")
        for i, row in recommended_books.iterrows():
            print(f"{i+1}. {row['title']}")
            if row['image']:
                display(Image(url=row['image'], width=100))
            else:
                print("No image available")

,title,description,image
0,To Kill a Mockingbird,"One of the best-loved stories of all time, To ...",https://covers.openlibrary.org/b/id/12606502-M...
1,To Kill a Mockingbird,"Tom Robinson, Mayella Ewell, Atticus and Scout...",https://covers.openlibrary.org/b/id/8410842-M.jpg
2,Cliffsnotes to kill a mockingbird,,https://covers.openlibrary.org/b/id/7145849-M.jpg
3,To Kill a Mockingbird,,https://covers.openlibrary.org/b/id/4800878-M.jpg
4,To kill a mockingbird,,None


Recommended Books:
2. To Kill a Mockingbird


11. To kill a mockingbird


17. Harper Lee's To kill a mockingbird


3. Cliffsnotes to kill a mockingbird


4. To Kill a Mockingbird


5. To kill a mockingbird
No image available
6. To Kill A Mockingbird [SparkNotes]


7. To kill a mockingbird


8. To Kill a Mockingbird


9. To kill a mockingbird
No image available
